In [10]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

db_config = {
    'user': 'practicum_student', # username
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',  # ← Línea completa
    'port': 5432, # connection port
    'db': 'data-analyst-final-project-db' # the name of the database
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


# 1. Objetivos del estudio
Queremos analizar la base de datos de libros y usuarios para responder:
1. Número de libros publicados después del 1 de enero de 2000.
2. Número de reseñas de usuarios y calificación promedio por libro.
3. Editorial con mayor número de libros con más de 50 páginas.
4. Autor con la calificación promedio más alta (mínimo 50 calificaciones).
5. Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

# 2. Revisar las tablas

In [2]:
# imprimir primeras filas de cada tabla
for table in ["books", "authors", "publishers", "ratings", "reviews"]:
    print(f"\nTabla: {table}")
    display(pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", con=engine))


Tabla: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Tabla: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Tabla: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Tabla: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Tabla: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# 3. Consultas SQL

### 3.1 Libros publicados después del 1 de enero de 2000

In [3]:

query = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""
pd.read_sql(query, con=engine)


,books_after_2000
0,819


### 3.2 Número de reseñas y calificación promedio por libro

In [4]:
query = """
SELECT b.title,
       COUNT(DISTINCT r.review_id) AS reviews_count,
       ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.title
ORDER BY avg_rating DESC NULLS LAST;
"""
pd.read_sql(query, con=engine)

,title,reviews_count,avg_rating
0,Dead Souls,2,5.00
1,How to Be a Domestic Goddess: Baking and the A...,1,5.00
2,Tai-Pan (Asian Saga #2),2,5.00
3,My Name Is Asher Lev,2,5.00
4,Marvel 1602,2,5.00
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,3,2.25
995,Drowning Ruth,3,2.00
996,Junky,2,2.00
997,His Excellency: George Washington,2,2.00


### 3.3 Editorial con más libros de más de 50 páginas

In [5]:
query = """
SELECT p.publisher,
       COUNT(b.book_id) AS books_count
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY books_count DESC
LIMIT 1;
"""
pd.read_sql(query, con=engine)

,publisher,books_count
0,Penguin Books,42


### 3.4 Autor con la calificación promedio más alta (mínimo 50 calificaciones)

In [8]:
query = """
SELECT a.author,
       ROUND(AVG(rt.rating), 2) AS avg_rating,
       COUNT(rt.rating_id) AS ratings_count
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY a.author
HAVING COUNT(rt.rating_id) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""
pd.read_sql(query, con=engine)

,author,avg_rating,ratings_count
0,Diana Gabaldon,4.3,50


### 3.5 Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros

In [9]:
query = """
WITH users_ratings AS (
    SELECT username,
           COUNT(DISTINCT book_id) AS books_rated
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
)
SELECT ROUND(AVG(reviews_count), 2) AS avg_reviews_per_user
FROM (
    SELECT ur.username,
           COUNT(rv.review_id) AS reviews_count
    FROM users_ratings ur
    LEFT JOIN reviews rv ON ur.username = rv.username
    GROUP BY ur.username
) sub;
"""
pd.read_sql(query, con=engine)

,avg_reviews_per_user
0,24.33


# 4. Conclusiones 

### Conclusiones
1. Hay 819 libros publicados después del año 2000.
2. Vimos el número de reseñas y el promedio de calificaciones por libro.
3. La editorial con más libros de más de 50 páginas es Penguin Books.
4. El autor con la mejor calificación promedio (mínimo 50 calificaciones) es Diana Gabaldon.
5. Los usuarios que calificaron más de 50 libros escriben en promedio 24.33 reseñas de texto.